# GPU-Accelerated Isogeometric Cloth Simulation on Watertight Surfaces

## Variational Formulation of Kirchhoff-Love Shells: A Computational Companion

### Supplementary Material for Chapter 4

**Rafael Cerqueira de Campos**  
Advisor: Prof. Dr. Afonso Paiva Neto  
Instituto de Ciencias Matematicas e de Computacao (ICMC)  
Universidade de Sao Paulo

---

*Interactive Derivation with Symbolic Verification*

This notebook provides a computational companion to the variational formulation derivation in Chapter 4 of the thesis. Rather than repeating the derivation verbatim, this notebook:

1. Uses SageMath to symbolically verify each mathematical identity
2. Provides additional intermediate steps that illuminate the mechanics
3. Offers interactive exploration of the relationships between quantities
4. Demonstrates numerical verification of symbolic results

The primary reference is:
Belytschko, T., Liu, W. K., Moran, B., & Elkhodary, K. (2014). *Nonlinear finite elements for continua and structures*. John Wiley & Sons.

## Initialization

First, we set up our symbolic environment and define conventions.

In [ ]:
# Define symbolic variables
var('rho h z')
assume(rho > 0)
assume(h > 0)
assume(z, 'real')

# Coordinate indices (for documentation)
print("Index Notation Conventions:")
print("  Greek indices (α, β, γ): range {1, 2} - Surface (in-plane) coordinates")
print("  Latin indices (i, j, k): range {1, 2, 3} - 3D spatial coordinates")
print("  Comma notation: partial derivative, e.g., v_{i,j} = ∂v_i/∂x_j")
print("  Repeated indices imply summation (Einstein convention).")

---
# Part I: From Strong Form to Weak Form

## 1. The Strong Form: Cauchy's Momentum Equation

We begin with Cauchy's equation of motion, which expresses conservation of linear momentum:

$$\frac{\partial \sigma_{ji}}{\partial x_j} + \rho b_i = \rho a_i \quad \text{in } \Omega$$

Physical interpretation of each term:
- $\sigma_{ji}$: Cauchy stress tensor (internal forces per unit area)
- $b_i$: Body force per unit mass (e.g., gravity)
- $a_i = dv_i/dt$: Acceleration (material derivative of velocity)
- $\rho$: Mass density (mass per unit volume)

In [ ]:
# Define symbolic functions for the derivation
var('x1 x2 x3')  # Spatial coordinates
x = [x1, x2, x3]

# Symbolic placeholders (we'll use these for structure, not computation)
var('sigma_11 sigma_12 sigma_13 sigma_21 sigma_22 sigma_23 sigma_31 sigma_32 sigma_33')
var('b_1 b_2 b_3 a_1 a_2 a_3')
var('dv_1 dv_2 dv_3')  # Virtual velocities

print("Symbolic placeholders defined for: σ, b, a, δv, x")
print("\nThe strong form equation holds pointwise in Ω.")
print("The derivation proceeds by converting this to a weak (integral) form.")

## 2. The Principle of Virtual Work

To convert the strong form to a weak form, we:

**Step 1:** Multiply by an arbitrary test function (virtual velocity) $\delta v_i$

**Step 2:** Integrate over the entire domain $\Omega$

$$\int_\Omega \delta v_i \left( \frac{\partial \sigma_{ji}}{\partial x_j} + \rho b_i - \rho a_i \right) d\Omega = 0$$

This is the principle of virtual work for linear momentum.

*Why this works: If the strong form equals zero at every point, then its integral weighted by any test function must also be zero. The converse is also true under mild regularity conditions.*

## 3. The Product Rule Identity

The key to deriving the weak form is the product rule. We want to move the derivative off the stress tensor and onto the test function.

**The Product Rule:**

$$\frac{\partial}{\partial x_j}(\delta v_i \sigma_{ji}) = \sigma_{ji} \frac{\partial \delta v_i}{\partial x_j} + \delta v_i \frac{\partial \sigma_{ji}}{\partial x_j}$$

Let's verify this symbolically:

In [ ]:
# Define symbolic functions with explicit spatial dependence
var('t')  # Can use as parameter
dv_func = function('dv')(x1, x2, x3)
sigma_func = function('sigma')(x1, x2, x3)

# Left side of product rule: d/dx_j (δv_i * σ_ji)
product = dv_func * sigma_func
left_side = diff(product, x1)  # Derivative with respect to x1 as example

# Right side: σ_ji * d(δv_i)/dx_j + δv_i * d(σ_ji)/dx_j
right_side = sigma_func * diff(dv_func, x1) + dv_func * diff(sigma_func, x1)

# Verify equality
verification = (left_side - right_side).simplify_full()

print("Left side:  ", left_side)
print("Right side: ", right_side)
print("Difference: ", verification, "(Verified!)" if verification == 0 else "(Error!)")

Rearranging to isolate the stress divergence term (which appears in Cauchy's equation):

$$\delta v_i \frac{\partial \sigma_{ji}}{\partial x_j} = \frac{\partial}{\partial x_j}(\delta v_i \sigma_{ji}) - \sigma_{ji} \frac{\partial \delta v_i}{\partial x_j}$$

## 4. The Divergence Theorem

Gauss's Divergence Theorem converts a volume integral of a divergence into a surface integral:

$$\int_\Omega \frac{\partial F_j}{\partial x_j} d\Omega = \int_\Gamma F_j n_j d\Gamma$$

where $n_j$ is the outward unit normal to the boundary $\Gamma$.

Applying to our problem with $F_j = \delta v_i \sigma_{ji}$:

$$\int_\Omega \frac{\partial}{\partial x_j}(\delta v_i \sigma_{ji}) d\Omega = \int_\Gamma \delta v_i \sigma_{ji} n_j d\Gamma$$

This is the crucial step that introduces the boundary integral, allowing us to incorporate boundary conditions naturally.

## 5. Boundary Conditions

The boundary $\Gamma$ is split into two complementary parts:

$$\Gamma = \Gamma_u \cup \Gamma_t, \quad \text{where } \Gamma_u \cap \Gamma_t = \emptyset$$

- $\Gamma_u$ (Dirichlet boundary): Displacement is prescribed, so $\delta v_i = 0$
- $\Gamma_t$ (Neumann boundary): Traction is prescribed: $t_i = \sigma_{ji} n_j$

Therefore, the boundary integral simplifies:

$$\int_\Gamma \delta v_i \sigma_{ji} n_j d\Gamma = \int_{\Gamma_t} \delta v_i t_i d\Gamma$$

## 6. The 3D Weak Form (Virtual Power Equation)

Substituting everything back and rearranging, we obtain the virtual power equation for a 3D continuum:

$$\int_\Omega \sigma_{ij} \frac{\partial \delta v_i}{\partial x_j} d\Omega = \int_\Omega \delta v_i \rho b_i d\Omega + \int_{\Gamma_t} \delta v_i t_i d\Gamma - \int_\Omega \delta v_i \rho a_i d\Omega$$

**Term-by-term interpretation:**
- $\delta P^{int}$ (left side): Internal virtual power (stress × virtual strain rate)
- $\delta P^{ext}$ (first two terms on right): External virtual power (body forces + tractions)
- $\delta P^{kin}$ (last term on right): Kinetic virtual power (inertia)

---
# Part II: Specialization to Kirchhoff-Love Shells

## 7. The Symmetric Stress Tensor

For a symmetric Cauchy stress tensor ($\sigma_{ij} = \sigma_{ji}$), we can relate the internal virtual power to the virtual strain rate.

**Velocity gradient decomposition:**

$$\frac{\partial \delta v_i}{\partial x_j} = \frac{1}{2}(\delta v_{i,j} + \delta v_{j,i}) + \frac{1}{2}(\delta v_{i,j} - \delta v_{j,i}) = \text{symmetric part} + \text{skew-symmetric part}$$

In [ ]:
# Define velocity gradient decomposition
dv1 = function('dv1')(x1, x2, x3)
dv2 = function('dv2')(x1, x2, x3)

def velocity_gradient(i, j, dv_funcs, coords):
    """Compute ∂(δv_i)/∂x_j"""
    return diff(dv_funcs[i], coords[j])

def symmetric_part(i, j, dv_funcs, coords):
    """Symmetric part: (1/2)(δv_{i,j} + δv_{j,i})"""
    return (velocity_gradient(i, j, dv_funcs, coords) + 
            velocity_gradient(j, i, dv_funcs, coords)) / 2

def skew_part(i, j, dv_funcs, coords):
    """Skew-symmetric part: (1/2)(δv_{i,j} - δv_{j,i})"""
    return (velocity_gradient(i, j, dv_funcs, coords) - 
            velocity_gradient(j, i, dv_funcs, coords)) / 2

# Verify decomposition for a 2D example
dv_funcs = [dv1, dv2]
coords = [x1, x2]

# Check that symmetric + skew = full gradient
i, j = 0, 1  # Example indices
full_grad = velocity_gradient(i, j, dv_funcs, coords)
sym = symmetric_part(i, j, dv_funcs, coords)
skew = skew_part(i, j, dv_funcs, coords)

decomposition_check = (full_grad - sym - skew).simplify_full()
print("Decomposition check (should be 0):", decomposition_check)

**Key result:** For symmetric $\sigma$, the skew-symmetric part does no work. We can show that $\sigma_{ij} W_{ij} = 0$ when $\sigma$ is symmetric and $W$ is skew-symmetric.

In [ ]:
# Demonstrate with 2x2 example
var('sigma11 sigma12 sigma22 W12')

# Symmetric stress tensor
sigma_sym = matrix([[sigma11, sigma12], 
                    [sigma12, sigma22]])

# Skew-symmetric tensor
W_skew = matrix([[0, W12], 
                 [-W12, 0]])

# Double contraction: σ_ij W_ij
double_contraction = sum(sigma_sym[i,j] * W_skew[i,j] 
                         for i in range(2) for j in range(2))

print("Double contraction σ_ij W_ij =", double_contraction.simplify_full())
print("(Skew part does no work for symmetric stress)")

Therefore, we can replace the velocity gradient with the virtual strain rate:

$$\delta \varepsilon_{ij} := \frac{1}{2}(\delta v_{i,j} + \delta v_{j,i})$$

and the internal virtual power becomes:

$$\delta P^{int} = \int_\Omega \sigma_{ij} \delta \varepsilon_{ij} d\Omega$$

## 8. The Kirchhoff-Love Kinematic Assumption

Now we specialize from a general 3D continuum to a thin shell.

**Shell Domain:** $\Omega = A \times (-h/2, h/2)$

**Kirchhoff-Love Assumptions:**
1. Normals remain straight and perpendicular to the deformed midsurface
2. No transverse shear deformation
3. No thickness stretching

**Consequence:** Strain varies linearly through the thickness:

$$\delta \varepsilon_{\alpha\beta}(x_\alpha, z) = \delta E_{\alpha\beta}(x_\alpha) + z \, \delta \kappa_{\alpha\beta}(x_\alpha)$$

where:
- $\delta E_{\alpha\beta}$ = membrane strain (midsurface stretching, constant in $z$)
- $\delta \kappa_{\alpha\beta}$ = curvature change (bending, linear in $z$)

In [ ]:
# Define Kirchhoff-Love strain decomposition
var('z h')
var('dE_ab dkappa_ab')  # Membrane strain and curvature (symbolic)

def delta_epsilon_shell(dE, dkappa, z):
    """Kirchhoff-Love strain: ε = E + z*κ"""
    return dE + z * dkappa

print("Kirchhoff-Love strain decomposition:")
print("  δε_{αβ}(z) =", delta_epsilon_shell(dE_ab, dkappa_ab, z))

print("\nAt different z positions:")
print("  z = 0 (midsurface):", delta_epsilon_shell(dE_ab, dkappa_ab, 0))
print("  z = h/2 (top):     ", delta_epsilon_shell(dE_ab, dkappa_ab, h/2))
print("  z = -h/2 (bottom): ", delta_epsilon_shell(dE_ab, dkappa_ab, -h/2))

## 9. Through-Thickness Integration of Internal Power

Now we integrate the internal virtual power through the shell thickness:

$$\delta P^{int} = \int_A \int_{-h/2}^{h/2} \sigma_{\alpha\beta} \, \delta \varepsilon_{\alpha\beta} \, dz \, dA$$

In [ ]:
print("=== Through-Thickness Integration ===")

# For CONSTANT stress through thickness
print("\n--- Case 1: Constant stress through thickness ---")
var('sigma0')

membrane_integral_const = integrate(sigma0, (z, -h/2, h/2))
bending_integral_const = integrate(z * sigma0, (z, -h/2, h/2))

print("  ∫σ₀ dz =", membrane_integral_const)
print("  ∫z σ₀ dz =", bending_integral_const, "(odd function -> zero)")

In [ ]:
# For LINEARLY varying stress through thickness
print("--- Case 2: Linearly varying stress (membrane + bending) ---")
var('sigma1')

def sigma_linear(z):
    return sigma0 + z * sigma1

membrane_integral_lin = integrate(sigma_linear(z), (z, -h/2, h/2))
bending_integral_lin = integrate(z * sigma_linear(z), (z, -h/2, h/2)).simplify_full()

print("  ∫(σ₀ + z σ₁) dz =", membrane_integral_lin)
print("  ∫z(σ₀ + z σ₁) dz =", bending_integral_lin)

**Defining the stress resultants:**

$$S := \frac{1}{h} \int_{-h/2}^{h/2} \sigma \, dz \quad \text{(membrane stress resultant)}$$

$$m := \int_{-h/2}^{h/2} z \, \sigma \, dz \quad \text{(bending moment resultant)}$$

Therefore, the internal virtual power for a Kirchhoff-Love shell becomes:

$$\delta P^{int} = \int_A \left( h \, \delta E^T S + \delta \kappa^T m \right) dA$$

## 10. Through-Thickness Integration of External Power

The external virtual power has contributions from:
- Body forces (e.g., gravity)
- Surface tractions (top/bottom faces)
- Edge tractions (lateral boundary)

In [ ]:
# Surface density integration
rho_s = integrate(rho, (z, -h/2, h/2))

print("Surface density: ρ_s = ∫ρ dz =", rho_s)
print("For constant density ρ, this gives ρ_s = ρh")

The combined external virtual power for a shell is:

$$\delta P^{ext} = \int_A \delta u^T (\rho b + f_d) \, dA + \int_{\partial A} (\delta u^T t) h \, ds$$

where $f_d := t^+ + t^-$ is the net distributed face load.

## 11. Kinetic Virtual Power

The inertial term follows the same pattern:

$$\delta P^{kin} = \int_\Omega \rho \, \delta u^T a \, d\Omega = \int_A \rho_s \, \delta u^T a \, dA$$

---
# Part III: The Complete Weak Form

## 12. The Virtual Power Equation for Kirchhoff-Love Shells

Combining all terms using $\delta P^{int} = \delta P^{ext} - \delta P^{kin}$ and rearranging:

---
### THE WEAK FORM: VIRTUAL POWER EQUATION FOR K-L SHELLS
---

$$\int_A \rho \, \delta u^T a \, dA + \int_A \left( h \, \delta E^T S + \delta \kappa^T m \right) dA = \int_A \delta u^T (\rho b + f_d) \, dA + \int_{\partial A} (\delta u^T t) h \, ds$$

**Term-by-term physical interpretation:**

**LEFT SIDE (must balance):**
- $\int_A \rho \, \delta u^T a \, dA$ → Inertial resistance (mass × acceleration)
- $\int_A h \, \delta E^T S \, dA$ → Membrane resistance (in-plane stretching)
- $\int_A \delta \kappa^T m \, dA$ → Bending resistance (curvature changes)

**RIGHT SIDE (applied loads):**
- $\int_A \delta u^T \rho b \, dA$ → Body forces (e.g., gravity)
- $\int_A \delta u^T f_d \, dA$ → Distributed surface loads
- $\int_{\partial A} (\delta u^T t) h \, ds$ → Edge tractions

**This is Equation 4.34 in Chapter 4 - the foundation for the cloth simulation framework.**

## 13. Summary of the Derivation

| Step | Action | Result |
|------|--------|--------|
| 1 | Start with Cauchy's momentum eq | Strong form (PDE) |
| 2 | Multiply by test fn, integrate | Principle of virtual work |
| 3 | Apply product rule | Isolate stress divergence |
| 4 | Apply divergence theorem | Introduce boundary integral |
| 5 | Apply boundary conditions | Split into Dirichlet/Neumann |
| 6 | Obtain 3D weak form | Virtual power equation |
| 7 | Use symmetric stress property | Replace with virtual strain |
| 8 | Apply K-L strain decomposition | $\varepsilon = E + z \kappa$ |
| 9 | Integrate through thickness | Define $S$ and $m$ |
| 10 | Combine all terms | Final shell weak form |

**Why this formulation?**

1. **Dimensionality reduction:** 3D problem → 2D (midsurface only)
2. **Natural boundary conditions:** Tractions incorporated automatically
3. **FEM compatibility:** Amenable to finite element discretization
4. **Clear separation:** Membrane and bending behaviors distinct
5. **IGA ready:** Forms basis for subdivision surface discretization

---
# Part IV: Numerical Verification

## 14. Verifying the Key Integrals

Let's verify our symbolic results with concrete numerical values:

In [ ]:
# Set numerical values for verification
h_num = 0.01       # thickness [m]
rho_num = 1000     # density [kg/m^3]
sigma0_num = 1000  # constant stress component [Pa]
sigma1_num = 50000 # linear stress gradient [Pa/m]

print("=== Numerical Verification ===")
print(f"Parameters: h = {h_num} m, ρ = {rho_num} kg/m³")

In [ ]:
# Verify membrane force resultant
print("\n--- Membrane Force Resultant ---")

membrane_direct = numerical_integral(lambda z: sigma0_num, -h_num/2, h_num/2)[0]
membrane_formula = sigma0_num * h_num

print(f"  Numerical:  {membrane_direct}")
print(f"  Formula:    {membrane_formula}")
print(f"  Match: {'Yes' if abs(membrane_direct - membrane_formula) < 1e-10 else 'No'}")

In [ ]:
# Verify bending moment
print("--- Bending Moment Resultant ---")

bending_direct = numerical_integral(
    lambda z: z * (sigma0_num + z * sigma1_num), 
    -h_num/2, h_num/2
)[0]
bending_formula = sigma1_num * h_num**3 / 12

print(f"  Numerical:  {bending_direct}")
print(f"  Formula:    {bending_formula}")
print(f"  Match: {'Yes' if abs(bending_direct - bending_formula) < 1e-10 else 'No'}")

In [ ]:
# Verify surface density
print("--- Surface Density ---")

rho_s_direct = numerical_integral(lambda z: rho_num, -h_num/2, h_num/2)[0]
rho_s_formula = rho_num * h_num

print(f"  Numerical:  {rho_s_direct} kg/m²")
print(f"  Formula:    {rho_s_formula} kg/m²")
print(f"  Match: {'Yes' if abs(rho_s_direct - rho_s_formula) < 1e-10 else 'No'}")

print("\n=== All verifications complete! ===")

## 15. Connection to IGA Implementation

*The weak form from this chapter forms the foundation for the isogeometric analysis (IGA) discretization discussed in Chapters 5-7.*

**IGA Discretization:**

Displacement interpolation:
$$u = N d$$
where $N$ = shape functions, $d$ = nodal displacements

Strain-displacement relations:
$$E = B_m d \quad \text{(membrane strain)}$$
$$\kappa = B_b d \quad \text{(bending strain/curvature)}$$

Semi-discrete equation of motion:
$$M a + f_{int}(d) = f_{ext}$$

where:
- $M = \int_A \rho N^T N \, dA$ (mass matrix)
- $f_{int} = \int_A (h B_m^T S + B_b^T m) \, dA$ (internal forces)
- $f_{ext} = \int_A N^T (\rho b + f_d) \, dA$ (external forces)

**Note:** The $B$ matrices (strain-displacement) require second derivatives of the shape functions due to the curvature term $\kappa$. This is why subdivision surfaces (which provide $C^1$ continuity) or NURBS are needed for IGA with Kirchhoff-Love shells.